In [1]:
import os
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
from rasterio import plot, mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import matplotlib.pyplot as plt
import rasterstats
import rioxarray as rxr
from matplotlib_scalebar.scalebar import ScaleBar
from pathlib import Path
import xarray as xr
from shapely.geometry import Point, Polygon
import xyzservices.providers as xyz
import contextily as ctx
import censusdata
import seaborn as sns
import requests
import zipfile
import shutil
import requests
import rasterio.features 
from shapely.geometry import shape

from rasterio.transform import from_origin
from scipy.spatial import cKDTree

import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

# Land Cover

In [ ]:
land_url = " href here "

land_response = requests.get(land_url)

if land_response.status_code == 200:
    with open("Data/land_use.tif", "wb") as f:
        f.write(land_response.content)
    print("Land Cover Data Downloaded!")
else:
    print(f"Failed to fetch land data: {land_response.status_code}")

# Radiation

In [2]:
#Code Aquired from Chat
# Define the ImageServer exportImage URL
server_url = "https://gis.earthdata.nasa.gov/image/rest/services/POWER/POWER_901_MONTHLY_RADIATION_UTC/ImageServer/exportImage"

#Spokane Box Boundary
# -117.823629	47.259272	-117.039763	48.047877	53063

# Define parameters for exporting an image
params = {
    "bbox": "-117.823629,47.259272,-117.039763,48.047877,53063",  # Bounding box covering Spokane County
    "bboxSR": "4326",
    "imageSR": "4326",
    "size": "2000,2000",  # Resolution of the image
    "format": "tiff",  # Use "png" if you prefer a simple image
    "f": "json",
}

# Request the image
response = requests.get(server_url, params=params)

if response.status_code == 200:
    image_data = response.json()
    image_url = image_data.get("href", None)  # Extract image URL

    if image_url:
        print(f"Downloading image from: {image_url}")

        # Download the raster image
        image_response = requests.get(image_url)
        image_filename = "Data/radiation.tif"

        with open(image_filename, "wb") as file:
            file.write(image_response.content)

        print(f"Image saved as {image_filename}")
    else:
        print("No image URL found in response.")
else:
    print(f"Failed to fetch data: {response.status_code}, Response: {response.text}")


Image saved as Data/radiation.tif


# Substations

In [ ]:
substation_url = "https://services6.arcgis.com/OO2s4OoyCZkYJ6oE/arcgis/rest/services/Substations/FeatureServer/0/query"

substation_params = {
    "where": "COUNTY = 'SPOKANE'", 
    "outFields": "*", 
    "f": "geojson" 
}

substation_response = requests.get(substation_url, params=substation_params)

if substation_response.status_code == 200:
    with open("Data/substations.geojson", "wb") as f:
        f.write(substation_response.content)
    print("Substation Data Downloaded!")
else:
    print(f"Failed to fetch substation data: {substation_response.status_code}")

# Substation Distance TIF

In [ ]:
# Code aquired from CHAT

substations = substation_gdf

tract_geom_gdf =tract_geom_gdf.to_crs(32610)
substations = substations.to_crs(epsg=32610)

# Get the bounding box from substations
xmin, ymin, xmax, ymax = tract_geom_gdf.total_bounds

# Define the resolution of the raster (in meters)
resolution = 100  # Change this as needed (e.g., 100 meters per pixel)
width = int((xmax - xmin) / resolution)
height = int((ymax - ymin) / resolution)


raster = np.full((height, width), np.nan, dtype=np.float32)

# Extract coordinates of substations
substation_coords = np.array(list(zip(substations.geometry.x, substations.geometry.y)))

# Create a spatial index using cKDTree for efficient distance computation
tree = cKDTree(substation_coords)

# Compute distances for each pixel
for i in range(height):
    for j in range(width):
        x = xmin + j * resolution + resolution / 2
        y = ymax - i * resolution - resolution / 2
        dist, _ = tree.query((x, y))
        raster[i, j] = dist / 1000  # converts it to km


transform = from_origin(xmin, ymax, resolution, resolution)

# Save raster to file
with rasterio.open(
    "./Data/distance_raster_proj_alt.tif", "w", driver="GTiff", height=height, width=width,
    count=1, dtype=np.float32, crs="EPSG:32610", transform=transform
) as dst:
    dst.write(raster, 1)


